In [0]:
!pip install https://github.com/matplotlib/mpl_finance/archive/master.zip
!pip install mpl-finance
import pandas as pd
from pandas import Series
import numpy as np
import requests
import csv
import matplotlib.pyplot as plt
import warnings
from matplotlib import dates
import matplotlib as mlp
from mpl_finance import candlestick_ohlc
%matplotlib inline

     - 747kB 9.1MB/s
  Created wheel for mpl-finance: filename=mpl_finance-0.10.0-cp36-none-any.whl size=6903 sha256=2275455e3ef3eb56a69c84efc53514a1548c45eccc8a91997ad3aa78a4a816ac
  Stored in directory: /tmp/pip-ephem-wheel-cache-x2dk3zj6/wheels/8f/bf/8a/6c858b2a3be945c7b2d658fbf6ab797b451d686cbee8aa2679
Successfully built mpl-finance


In [0]:
#Task 1: Choose one or more suitable web API(s)


**API Selected:**

Choosen API for data acquisition provided by https://www.quandl.com/
Specifically, the data related to stock prices, dividends and splits API - https://www.quandl.com/databases/WIKIP

Quandl API is freely available upon signing up.

The API key required for the extration of data is f8zeBpAmA6dyyFdomf91



In [0]:
#Retrieve data from the chosen API
#Function to request data from the url provided using the 'companies' argument and the global variable 'apikey' 
def get_raw_data(companies):
    url = "https://www.quandl.com/api/v3/datatables/WIKI/PRICES.csv?ticker="+companies["Alphabet"]+"&&api_key="+apikey
    get_response = requests.get(url) #Requests to get data from url
    if get_response.status_code != 200: #Retrieval Error Response
        print("Unable to retrive data",get_response.status_code)
    else:
        print("Data Retrieved & Status Code:",get_response.status_code)
        data = csv.reader(get_response.text.strip().split('\n')) #Splits the data on the basis of newline and stores the result
        return data

In [0]:
apikey = "f8zeBpAmA6dyyFdomf91"       #API KEY retrieved on Signup

In [0]:
#Collected raw data, and Storing it in a File
#Variables to get the Raw Data from the API and Store in the File
companies = {"Alphabet":"GOOG","Microsoft":"MSFT","Apple":"AAPL"} #Dictionary of companies created for better understanding of code
store_data = get_raw_data(companies) #Calling the function and storing the value(list) 
with open('Raw_data.csv','w',newline='')as rawfile: #Creating a CSV File 
            w = csv.writer(rawfile)
            for rows in store_data: #Iterating over the retrieved List of CSV data
                w.writerow(rows) #Writing each row in the Raw_data.csv file created
            rawfile.close()
print('Raw File Saved')

Data Retrieved & Status Code: 200
Raw File Saved


In [0]:
#Load and Represent the data using Pandas DataFrame.
#Apply pre-processing steps to clean and filter the data.
#Parsing Raw Data:

#The raw data returns approx. 1000 lines of values, consisting stock data of a company Alphabet(GOOG)
#Selection of appropriate columns is needed.
collected_data = pd.read_csv('Raw_data.csv') #Loading the CSV File using Pandas DataFrame
collected_data.head(1000) #Preview of the Dataset

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
0,GOOG,2018-03-27,1063.00,1064.84,996.92,1005.10,3029471.0,0.0,1.0,1063.00,1064.84,996.92,1005.10,3029471.0
1,GOOG,2018-03-26,1046.00,1055.63,1008.40,1053.21,2558385.0,0.0,1.0,1046.00,1055.63,1008.40,1053.21,2558385.0
2,GOOG,2018-03-23,1047.03,1063.36,1021.22,1021.57,2113497.0,0.0,1.0,1047.03,1063.36,1021.22,1021.57,2113497.0
3,GOOG,2018-03-22,1081.88,1082.90,1045.91,1049.08,2580374.0,0.0,1.0,1081.88,1082.90,1045.91,1049.08,2580374.0
4,GOOG,2018-03-21,1092.74,1106.30,1085.15,1090.88,1640709.0,0.0,1.0,1092.74,1106.30,1085.15,1090.88,1640709.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,GOOG,2014-04-11,532.55,540.00,526.53,530.60,3914100.0,0.0,1.0,532.55,540.00,526.53,530.60,3914100.0
996,GOOG,2014-04-10,565.00,565.00,539.90,540.95,4025800.0,0.0,1.0,565.00,565.00,539.90,540.95,4025800.0
997,GOOG,2014-04-09,559.62,565.37,552.95,564.14,3321700.0,0.0,1.0,559.62,565.37,552.95,564.14,3321700.0
998,GOOG,2014-04-08,542.60,555.00,541.61,554.90,3142600.0,0.0,1.0,542.60,555.00,541.61,554.90,3142600.0


In [0]:
#Using the Pandas Library for additional Data Modification



*   Above is the preview of the dataset (First Five Records).
*   8 columns are chosen for analysis, namely ticker, date, open, high, low, close, volume and adj_low for each day



In [0]:
#Converting date field to DateTime
collected_data['date'] = pd.to_datetime(collected_data['date'])
#Selecting on the desired columns for Analysis
selected_data = collected_data.loc[:,['ticker','date','open','high','low','close','volume','adj_close']]
#Extracting/Filtering data from the year 2016 and untill now (last date 16-March-2018)
#Adding a column Year, extracting the year from Date column
selected_data['Year'] = selected_data['date'].apply(lambda x:x.strftime('%Y'))
#Renaming the column name ticker to Company
selected_data = selected_data.rename(columns={'ticker':'Company'})
#Previewing the First 5 Records
selected_data.head()

,Company,date,open,high,low,close,volume,adj_close,Year
0,GOOG,2018-03-27,1063.00,1064.84,996.92,1005.10,3029471.0,1005.10,2018
1,GOOG,2018-03-26,1046.00,1055.63,1008.40,1053.21,2558385.0,1053.21,2018
2,GOOG,2018-03-23,1047.03,1063.36,1021.22,1021.57,2113497.0,1021.57,2018
3,GOOG,2018-03-22,1081.88,1082.90,1045.91,1049.08,2580374.0,1049.08,2018
4,GOOG,2018-03-21,1092.74,1106.30,1085.15,1090.88,1640709.0,1090.88,2018


In [0]:
#Working on Missing Data
#Calculates the sum of all the null values in all the fields(columns)
selected_data.isnull().sum()

Company      0
date         0
open         0
high         0
low          0
close        0
volume       0
adj_close    0
Year         0
dtype: int64

In [0]:
#Further check if any of the numeric data is not string
selected_data.dtypes.value_counts()

float64           6
object            2
datetime64[ns]    1
dtype: int64

In [0]:
!apt install python-confluent-kafka

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  librdkafka1
Suggested packages:
  python-avro python-requests
The following NEW packages will be installed:
  librdkafka1 python-confluent-kafka
0 upgraded, 2 newly installed, 0 to remove and 7 not upgraded.
Need to get 323 kB of archives.
After this operation, 999 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 librdkafka1 amd64 0.11.3-1build1 [293 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-confluent-kafka amd64 0.11.0-1 [30.2 kB]
Fetched 323 kB in 1s (285 kB/s)
Selecting previously unselected package librdkafka1:amd64.
(Reading database ... 135004 files and directories currently installed.)
Preparing to unpack .../librdk

In [0]:
!pip install confluent-kafka
!pip install avro-python3

from confluent_kafka import avro
from confluent_kafka.avro import AvroProducer
import csv

  Created wheel for avro-python3: filename=avro_python3-1.9.1-cp36-none-any.whl size=43199 sha256=d0b0ca77692ab775fa8839e25da51d0d2b366649bad89887d62d26fcca6a1122
  Stored in directory: /root/.cache/pip/wheels/94/54/6f/a5df680fd3224aa45145686f3b1b02a878a90ea769fcf9daaf
Successfully built avro-python3
